In [6]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('CHAT_COMPLETION_NAME')
SEED=123

# temperature

Defaults to 1

What sampling temperature to use. Higher values means the model will take more risks. Try 0.9 for more creative applications, and 0 (argmax sampling) for ones with a well-defined answer.

We generally recommend altering this or top_p but not both.

In [13]:
def call_openai(num_times, prompt, temperature=0.75, use_seed=False):
    for i in range(num_times):
        if use_seed:
            response = client.chat.completions.create(
                model=CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    seed=SEED,
                    temperature = temperature
            )
        else:
            response = client.chat.completions.create(
                model=CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    temperature = temperature
            )
        print(response.choices[0].message.content)

In [14]:
# Without seed and temperature, the response is different each time
call_openai(10, 'The best pet is a ')

It really depends on your lifestyle and preferences. Some people love the companionship of dogs, while others prefer the independence of cats. Others may enjoy the low maintenance of small pets like fish or birds. It's important to consider your living situation, schedule, and the level of care and attention you can
The best pet is a personal preference and can depend on individual circumstances, lifestyle, and preferences. Some people may find dogs to be the best pet for their active lifestyle and desire for companionship, while others may prefer the low-maintenance nature of cats or the unique companionship of smaller animals like rabbits
The best pet is a subjective choice and varies from person to person. Some people may prefer dogs for their loyalty and companionship, while others may prefer cats for their independence and low maintenance. Other popular pets include birds, fish, rabbits, and reptiles. It ultimately depends on your lifestyle, living
The best pet is subjective and d

In [15]:
# Now using a seed and 0 temperature, the response is the much more consisitent
call_openai(10, 'The best pet is a ', temperature = 0, use_seed=True)

The best pet is a subjective choice and depends on individual preferences, lifestyle, and circumstances. Some people may prefer dogs for their loyalty and companionship, while others may prefer cats for their independence and low maintenance. Other popular pets include birds, fish, rabbits, and small mammals. It's important to
The best pet is a subjective choice and depends on individual preferences, lifestyle, and living situation. Some people may prefer dogs for their loyalty and companionship, while others may prefer cats for their independence and low maintenance. Other popular pets include birds, fish, rabbits, and reptiles. It's important
The best pet is a matter of personal preference and lifestyle. Some people may prefer dogs for their loyalty and companionship, while others may prefer cats for their independence and low maintenance. Other popular pets include birds, fish, rabbits, and reptiles. It's important to consider your living situation, schedule
The best pet is a matter

# n

Defaults to 1

How many completions to generate for each prompt.

Note: Because this parameter generates many completions, it can quickly consume your token quota. Use carefully and ensure that you have reasonable settings for max_tokens and stop.

In [17]:
response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages = [{"role":"system", "content":"You are a helpful assistant."},
                        {"role":"user","content": "The best pet is a "}],
                max_tokens=60,
                n=2
        )

for c in response.choices:
    print(c.message.content)

The best pet for someone can vary depending on their lifestyle, preferences, and living situation. Some people might prefer a dog for companionship and outdoor activities, while others might prefer a cat for low-maintenance companionship. Other options, such as birds, fish, or small mammals, could also make
It depends on your lifestyle and what you're looking for in a pet. Some people might prefer dogs for their loyalty and companionship, while others might prefer cats for their independence and low-maintenance care. Fish, birds, reptiles, and small mammals are also popular choices for different reasons. It


# presence_penalty

Defaults to 0

Number between -2.0 and 2.0. Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.

# frequency_penalty

Defaults to 0

Number between -2.0 and 2.0. Positive values penalize new tokens based on their existing frequency in the text so far, decreasing the model's likelihood to repeat the same line verbatim.
